In [1]:
%load_ext autoreload
%load_ext tensorboard
%matplotlib inline

In [2]:
import matplotlib
import numpy as np
import os
import random
import yaml
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rc
from matplotlib import cm
import seaborn as sns
from importlib import reload
from pathlib import Path
import sklearn
import joblib
import torch
import pandas as pd
import copy

In [3]:
CUB_DIR = "/homes/me466/data/CUB200/"
CUB_IMAGE_DIR = os.path.join(CUB_DIR, "CUB_200_2011/images/")
CUB_IMAGE_PATH_FILE = os.path.join(CUB_DIR, "CUB_200_2011/images.txt")

with open(CUB_IMAGE_PATH_FILE, 'r') as f:
    cub_image_paths = [x.rstrip().split(" ")[-1] for x in f]
print("Found a total of", len(cub_image_paths), "CUB images")

Found a total of 11788 CUB images


In [4]:
# READ ATTRIBUTES

def attr_line_to_val(line):
    return int(line.split(" ")[2])

def get_sample_attributes(idx, lines):
    return np.array([
        attr_line_to_val(lines[x]) for x in range(idx * 312, (idx+1) * 312)
    ])

with open(
    os.path.join(
        CUB_DIR,
        'CUB_200_2011/attributes/image_attribute_labels.txt',
    ),
    'r',
) as f:
    lines = [x.rstrip() for x in f]
attributes = np.array([
    get_sample_attributes(idx, lines) for idx in range(len(cub_image_paths))
])
print("Found a total of", attributes.shape, "CUB attributes")


Found a total of (11788, 312) CUB attributes


In [5]:
import clip
device = 'cpu'

# Load VLM
model, preprocess = clip.load("ViT-B/32", device=device)

positive_concept_embeddings = {}
negative_concept_embeddings = {}

In [6]:
# Load CUB Images onto memory to speed things up
from PIL import Image
import random
random.seed(42)

dataset_size = int(np.ceil(len(cub_image_paths) * 0.25))
print("Using", dataset_size, "of images...")
used_indices = random.sample(range(len(cub_image_paths)), dataset_size)
cub_images = [
    preprocess(
        Image.open(os.path.join(CUB_DIR, f"CUB_200_2011/images/{cub_image_paths[x]}"))
    ).unsqueeze(0).to(device)
    for x in used_indices
]
print(f"Working with {len(cub_images)} images!")

Using 2947 of images...
Working with 2947 images!


In [7]:
all_images = torch.concat(cub_images, 0)
codes = model.encode_image(all_images)

: 

: 

In [8]:
# Now precompute the images embeddings
cub_selected_embeddings = [codes[i:i+1, :, :, :] for i in range(codes.shape[0])]
# cub_selected_embeddings = [
#     model.encode_image(x)
#     for x in cub_images
# ]

: 

: 

In [ ]:
# Normalize the embeddings
cub_selected_embeddings = [
    x / x.norm(dim=1, keepdim=True)
    for x in cub_selected_embeddings
]

In [ ]:
# Compute zero-shot accuracy of embeddings for eachb concept across the entire training set
from cem.data.CUB200.cub_loader import CLASS_NAMES, CONCEPT_SEMANTICS, CONCEPT_SEMANTICS_SENTENCE, CONCEPT_SEMANTICS_NEGATION
import tqdm

from sklearn.metrics import roc_auc_score

with torch.no_grad():
    concept_mean_aucs = []
    for concept_idx, (positive_concept_dscr, negative_concept_dscr) in enumerate(zip(CONCEPT_SEMANTICS_SENTENCE, CONCEPT_SEMANTICS_NEGATION)):
        print(f"[Processing concept {concept_idx}/{len(CONCEPT_SEMANTICS_NEGATION)}]")
        print(f"\tGenerating concept embedding for positive concept '{positive_concept_dscr}'")
        if concept_idx not in positive_concept_embeddings:
            pos_text = clip.tokenize([positive_concept_dscr]).to(device)
            pos_emb = model.encode_text(pos_text)
            pos_emb = pos_emb / pos_emb.norm(dim=1, keepdim=True)
            positive_concept_embeddings[concept_idx] = pos_emb
        else:
            pos_emb = positive_concept_embeddings[concept_idx]

        # print(f"\tGenerating concept embedding for negative concept '{negative_concept_dscr}'")
        if concept_idx not in negative_concept_embeddings:
            neg_text = clip.tokenize([negative_concept_dscr]).to(device)
            neg_emb = model.encode_text(neg_text)
            neg_emb = neg_emb / neg_emb.norm(dim=1, keepdim=True)
            negative_concept_embeddings[concept_idx] = neg_emb
        else:
            neg_emb = negative_concept_embeddings[concept_idx]

        # print(f"\tCosine similarity between the two embeddings is {pos_emb @ neg_emb.t()}")
        # print(f"\tComputing predictive accuracy accross CUB samples")
        # Now time to compute their zero-shot scores on every CUB sample
        concept_preds = []
        ground_truth_vals = []
        for used_idx in tqdm.tqdm(range(len(used_indices))):
            #  Get the image embbedding
            image_emb = cub_selected_embeddings[used_idx]

            # Get its ground truth concept value
            image_idx = used_indices[used_idx]
            concept_val = attributes[image_idx, concept_idx]
            ground_truth_vals.append(concept_val)

            # Compute its positive and negative cosine similarities
            score_pos = pos_emb @ image_emb.t()
            score_neg = neg_emb @ image_emb.t()

            # And get a label
            zero_shot_preds = (model.logit_scale.exp() * torch.concat([score_neg, score_pos], dim=0)).softmax(dim=0).cpu().numpy()
            zero_shot_label = int(zero_shot_preds[1] >= 0.5)
            concept_preds.append(zero_shot_label)

        # And time to compute the actual concept accuracy
        concept_preds = np.array(concept_preds)
        ground_truth_vals = np.array(ground_truth_vals)
        # acc = np.mean(concept_preds == ground_truth_vals)
        try:
            auc = roc_auc_score(ground_truth_vals, concept_preds)
            include = True
        except:
            print(f"\tWARNING: all same labels for concept '{positive_concept_dscr}'")
            auc = np.mean(concept_preds == ground_truth_vals)
            include = False

        print(f"\tROC-AUC of predicting ground truth concept {concept_idx} using zero-shot classifier is: {100 * auc:.2f}%")
        concept_mean_aucs.append((concept_idx, auc, include))



[Processing concept 0/312]
	Generating concept embedding for positive concept 'a bird with a curved bill'
	Generating concept embedding for negative concept 'a bird whose bill is not a curved bill'
	Cosine similarity between the two embeddings is tensor([[0.9522]])
	Computing predictive accuracy accross CUB samples


100%|██████████| 236/236 [00:00<00:00, 20982.63it/s]


	ROC-AUC of predicting ground truth concept 0 using zero-shot classifier is: 59.64%
[Processing concept 1/312]
	Generating concept embedding for positive concept 'a bird with a dagger bill'
	Generating concept embedding for negative concept 'a bird whose bill is not a dagger bill'
	Cosine similarity between the two embeddings is tensor([[0.9620]])
	Computing predictive accuracy accross CUB samples


100%|██████████| 236/236 [00:00<00:00, 22249.95it/s]


	ROC-AUC of predicting ground truth concept 1 using zero-shot classifier is: 52.63%
[Processing concept 2/312]
	Generating concept embedding for positive concept 'a bird with a hooked bill'
	Generating concept embedding for negative concept 'a bird whose bill is not a hooked bill'
	Cosine similarity between the two embeddings is tensor([[0.9529]])
	Computing predictive accuracy accross CUB samples


...

100%|██████████| 236/236 [00:00<00:00, 22454.36it/s]

	ROC-AUC of predicting ground truth concept 310 using zero-shot classifier is: 51.04%
[Processing concept 311/312]
	Generating concept embedding for positive concept 'a bird with a multi-colored wing pattern'


	Generating concept embedding for negative concept 'a bird whose wings are not multi-colored'
	Cosine similarity between the two embeddings is tensor([[0.9180]])
	Computing predictive accuracy accross CUB samples


100%|██████████| 236/236 [00:00<00:00, 22443.67it/s]

	ROC-AUC of predicting ground truth concept 311 using zero-shot classifier is: 50.12%


In [ ]:
np.max([x[1] for x in concept_mean_aucs])

0.9915254237288136